In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

data = pd.read_csv('data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day_of_week'] = data['Date'].dt.dayofweek
data['Day'] = data['Date'].dt.day


X = data[['Day_of_week', 'Day', 'Month', 'Year','Event',
          'India_Reservation', 'Chinese > 100', 'Indian > 100']]
y = data[['Customers_Chinese', 'Customers_India']]


X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 7)

model = DecisionTreeRegressor(random_state=42)
model.fit(X_train,y_train)

predictions = pd.DataFrame(model.predict(X_test), columns=['Predicted_Customers_Chinese', 'Predicted_Customers_India'])
X_test['Date'] = X_test.apply(lambda row: f"{row['Day']}/{row['Month']}/{row['Year']}", axis=1)
predictions['Date']= pd.to_datetime(X_test['Date'].values, format='%d/%m/%Y')
predictions['Day_of_week'] = predictions['Date'].dt.day_name()
predictions = predictions[['Date', 'Day_of_week', 'Predicted_Customers_Chinese', 'Predicted_Customers_India']]

print(predictions)
